In [10]:
!pip install --quiet mediapipe moviepy Pillow

In [ ]:
import os, cv2, numpy as np, math, random
import mediapipe as mp
from PIL import Image, ImageOps, ImageDraw
from moviepy.editor import ImageClip, concatenate_videoclips, AudioFileClip
import gradio as gr

# === Init face detection ===
mp_face_mesh = mp.solutions.face_mesh.FaceMesh(
    static_image_mode=True,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5
)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

validated_points = {}  # {filename: [(x_left,y_left), (x_right,y_right)]}
current_file = None
clicked_points = []

# --- 1. Auto detect eyes ---
def detect_face_and_eyes(image_rgb):
    h, w, _ = image_rgb.shape
    results = []
    try:
        res = mp_face_mesh.process(cv2.cvtColor(image_rgb, cv2.COLOR_BGR2RGB))
        if res and res.multi_face_landmarks:
            lm = res.multi_face_landmarks[0].landmark
            left_eye = np.mean([(lm[33].x*w, lm[33].y*h), (lm[133].x*w, lm[133].y*h)], axis=0)
            right_eye = np.mean([(lm[362].x*w, lm[362].y*h), (lm[263].x*w, lm[263].y*h)], axis=0)
            results.append({"left_eye": left_eye, "right_eye": right_eye, "confidence": 0.95})
    except:
        pass
    try:
        gray = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 5, minSize=(100,100))
        if len(faces)>0:
            fx, fy, fw, fh = max(faces, key=lambda f: f[2]*f[3])
            cx = fx+fw/2; eye_y = fy+fh*0.37; sep = fw*0.25
            results.append({"left_eye":[cx-sep,eye_y], "right_eye":[cx+sep,eye_y], "confidence":0.75})
    except:
        pass
    if not results: return None
    return max(results, key=lambda x:x["confidence"])

# --- 2. Preview auto detection ---
def preview_auto(file):
    global current_file, clicked_points
    current_file = file.name
    clicked_points = []
    pil_img = Image.open(file.name).convert("RGB")
    img_np = np.array(pil_img)
    det = detect_face_and_eyes(img_np)
    draw = ImageDraw.Draw(pil_img)
    auto_points = []
    if det:
        auto_points = [(int(det["left_eye"][0]), int(det["left_eye"][1])),
                       (int(det["right_eye"][0]), int(det["right_eye"][1]))]
        for p in auto_points:
            draw.ellipse([p[0]-5,p[1]-5,p[0]+5,p[1]+5], outline="red", width=3)
    return pil_img, str(auto_points)

# --- 3. Handle clicks on image ---
def on_click(evt: gr.SelectData):
    global clicked_points, current_file
    clicked_points.append(evt.index)
    if len(clicked_points) > 2:
        clicked_points = clicked_points[-2:]
    return str(clicked_points)

# --- 4. Validate selection ---
def validate_points():
    global current_file, clicked_points, validated_points
    if current_file and len(clicked_points)==2:
        validated_points[current_file] = clicked_points.copy()
        return f"✅ {os.path.basename(current_file)} valid with {clicked_points}"
    return "⚠️ Please select exactly 2 points (left & right eyes)."

# --- 5. Align image ---
def align_eyes_to_fixed_position(image_rgb, eyes, target_left, target_right, output_size=(1024,768)):
    left, right = np.array(eyes[0]), np.array(eyes[1])
    center = (left+right)/2.0
    t_center = (np.array(target_left)+np.array(target_right))/2.0
    angle = math.atan2((right-left)[1], (right-left)[0])
    t_angle = math.atan2((target_right-target_left)[1], (target_right-target_left)[0])
    rot = t_angle-angle
    scale=1.0
    h,w=image_rgb.shape[:2]
    M=cv2.getRotationMatrix2D(tuple(center), math.degrees(rot), scale)
    rotated=cv2.warpAffine(image_rgb,M,(w,h),flags=cv2.INTER_LANCZOS4,borderValue=(255,255,255))
    new_c=np.dot(M,[center[0],center[1],1])
    trans=t_center-new_c
    M2=np.float32([[1,0,trans[0]],[0,1,trans[1]]])
    return cv2.warpAffine(rotated,M2,output_size,flags=cv2.INTER_LANCZOS4,borderValue=(255,255,255))

# --- 6. Slideshow ---
def create_slideshow(files, music=None, dur=4.0, fade=2.0):
    os.makedirs("outputs", exist_ok=True)
    clips=[]; out_size=(1024,768)
    cx=out_size[0]//2; eye_y=out_size[1]//3; sep=140
    FIXED_L=(cx-sep//2,eye_y); FIXED_R=(cx+sep//2,eye_y)
    for f in files:
        if f.name not in validated_points: continue
        pil_img=Image.open(f.name).convert("RGB")
        white_bg=Image.new("RGB", pil_img.size,(255,255,255)); white_bg.paste(pil_img,(0,0))
        bordered=ImageOps.expand(white_bg,border=30,fill="white")
        aligned=align_eyes_to_fixed_position(np.array(bordered), validated_points[f.name], FIXED_L,FIXED_R,out_size)
        rotation=random.uniform(-4,4)
        clip=ImageClip(aligned).set_duration(dur).rotate(rotation)
        clips.append(clip)
    if not clips: return "❌ No validated images", None, None
    final=concatenate_videoclips(clips,method="compose",padding=-fade).crossfadein(fade)
    if music:
        try:
            audio=AudioFileClip(music.name).set_duration(final.duration)
            final=final.set_audio(audio)
        except: pass
    out="outputs/final_slideshow.mp4"
    final.write_videofile(out,fps=24,codec="libx264",audio_codec="aac",
                          ffmpeg_params=["-pix_fmt","yuv420p"],verbose=False,logger=None)
    final.close()
    return f"✅ Video created with {len(clips)} validated images", out, out

# === Gradio UI ===
with gr.Blocks() as demo:
    gr.Markdown("## 👁️ Validate eyes before slideshow")
    with gr.Row():
        file_in=gr.File(file_types=["image"], file_count="multiple", label="📸 Photos")
        music_in=gr.File(file_types=["audio"], label="🎵 Music (optional)")
    preview_btn=gr.Button("🔍 Preview first photo")
    img=gr.Image(type="pil", interactive=True, label="Click 2 points (eyes)")
    log=gr.Textbox(label="Log")
    preview_btn.click(preview_auto, inputs=[file_in], outputs=[img, log])
    img.select(on_click, outputs=log)
    validate_btn=gr.Button("✅ Validate this photo")
    validate_btn.click(validate_points, outputs=log)
    dur=gr.Slider(2,8,4,0.5,label="Duration per image (s)")
    fade=gr.Slider(1,4,2,0.5,label="Fade duration (s)")
    gen_btn=gr.Button("🎬 Generate video")
    out_txt=gr.Textbox()
    out_vid=gr.Video()
    out_file=gr.File()
    gen_btn.click(create_slideshow, inputs=[file_in,music_in,dur,fade], outputs=[out_txt,out_vid,out_file])

demo.launch(share=True, debug=True)


